In [1]:
# load the necessary packages
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

import os
import numpy as np
import matplotlib.pyplot as plt

### Load Data and Model

In [9]:
batch_size = 30
IMG_HEIGHT = 180
IMG_WIDTH = 180
test_dir = 'data/resized/test'
dropout_rate = 0.15

In [11]:
test_image_generator = ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=test_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 25250 images belonging to 101 classes.


In [4]:
# load up saved model

model = load_model('tmp/models/CNN_MODEL_V1.h5')

In [5]:
model.variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
 array([[[[ 0.06282678,  0.04829698, -0.01844806, ..., -0.04746234,
           -0.0670765 ,  0.06795672],
          [-0.10898515, -0.05589448,  0.11402639, ...,  0.01933534,
           -0.05823009, -0.04042029],
          [ 0.15118654,  0.00703375, -0.00501012, ..., -0.00178792,
           -0.0083088 ,  0.12529804]],
 
         [[ 0.08833876,  0.13090399,  0.02507343, ..., -0.05661278,
            0.08365681,  0.17206325],
          [-0.00669257, -0.11730769,  0.1750399 , ..., -0.07019983,
            0.08646747,  0.01553954],
          [ 0.15031612,  0.15174249,  0.06245387, ..., -0.11878727,
           -0.06131674, -0.05036164]],
 
         [[-0.02588237,  0.08353666,  0.03758258, ..., -0.04442456,
           -0.02666054,  0.15065701],
          [ 0.01209998, -0.12866215,  0.09541449, ...,  0.06139058,
            0.07636137, -0.16358641],
          [ 0.19368741, -0.0165242 ,  0.09749999, ..., -0.07800639,
    

In [23]:
test_data_gen.classes

array([  0,   0,   0, ..., 100, 100, 100])

In [26]:
probabilities = model.predict_generator(test_data_gen,
                       steps=100)

In [27]:
y_pred = np.argmax(probabilities, axis=1)

In [28]:
y_pred

array([87, 44, 82, ..., 57, 99, 50], dtype=int64)